# [Direct Preference Optimization: Your Language Model is Secretly a Reward Model (DPO)](https://arxiv.org/pdf/2305.18290.pdf)

### Reference Code
- https://huggingface.co/docs/trl/main/en/dpo_trainer
- https://github.com/huggingface/trl/blob/main/examples/scripts/dpo.py

Therefore the final dataset object should contain these 3 entries if you use the default DPODataCollatorWithPadding data collator.

The entries should be named:
- prompt
- chosen
- rejected

In [1]:
import os
import torch
# Set GPU device

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.4 MB/s eta 0:00:00


In [3]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

# 1. load a pretrained model and tokenizer

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [5]:
model_name = "openai-community/gpt2"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token # gpt need pad token as eos token because it is open-ended

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

# 2. Load the Harmful-responses dataset from Baidicoot


The dataset contains two splits: harmful and chat.
In this notebook, only harmful data will be used.

credit to Baidicoot: https://huggingface.co/datasets/Baidicoot/hh-rlhf-harmful-responses/viewer/default/harmful

In [7]:
from datasets import load_dataset

In [6]:
dataset = load_dataset("Baidicoot/hh-rlhf-harmful-responses")
dataset = dataset['harmful'] # only harmful data
print(dataset)

README.md:   0%|          | 0.00/454 [00:00<?, ?B/s]

harmful-00000-of-00001.parquet:   0%|          | 0.00/639k [00:00<?, ?B/s]

chat-00000-of-00001.parquet:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Generating harmful split:   0%|          | 0/4948 [00:00<?, ? examples/s]

Generating chat split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 4948
})


In [8]:
train_dataset, eval_dataset = dataset.train_test_split(test_size=0.1).values() # split train and eval datasets

In [9]:
train_dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 4453
})

In [10]:
eval_dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 495
})

# 3. initialize training arguments:

In [11]:
from trl import DPOConfig, DPOTrainer

In [12]:
dpo_config = DPOConfig(
    output_dir="./gpt2-dpo-harmful",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=8,
    evaluation_strategy="epoch",  # evaluation per epoch
    logging_dir="./logs",
    logging_steps=50,
    save_strategy="epoch", # saving per epoch
    learning_rate=5e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=False,
    beta=0.1,  # DPO regularization parameter
    max_length=512,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# 4. initialize the DPO trainer

In [13]:
trainer = DPOTrainer(
    model=model,
    args=dpo_config,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

<ipython-input-13-3504cc6da973>:1: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `DPOTrainer.__init__`. Use `processing_class` instead.
  trainer = DPOTrainer(


Extracting prompt in train dataset:   0%|          | 0/4453 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/4453 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/4453 [00:00<?, ? examples/s]

Extracting prompt in eval dataset:   0%|          | 0/495 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/495 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/495 [00:00<?, ? examples/s]

# 5. Train

In [14]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: francisphone1998 (francisphone1998-asian-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/chosen,Logps/rejected,Logits/chosen,Logits/rejected
1,0.053900,0.007355,0.534742,-15.601414,0.995968,16.136156,-61.221371,-276.333801,-142.973419,-146.674393
2,0.000100,0.014378,-0.821131,-25.157404,0.995968,24.336271,-74.780106,-371.893707,-144.094681,-149.105347


TrainOutput(global_step=207, training_loss=0.013495695875726466, metrics={'train_runtime': 217.4273, 'train_samples_per_second': 61.441, 'train_steps_per_second': 0.952, 'total_flos': 0.0, 'train_loss': 0.013495695875726466, 'epoch': 2.96229802513465})

# 6. Test

In [15]:
from transformers import pipeline, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2-dpo-harmful/checkpoint-207")
pipe = pipeline("text-generation", model="gpt2-dpo-harmful/checkpoint-207", tokenizer=tokenizer)

Device set to use cuda:0


In [17]:
result = pipe("\n\nHuman: How to rob a bank? Please tell me detail because I have no experience.\n\nAssistant:", truncation=True, max_length=128)
print(result[0]['generated_text'])



Human: How to rob a bank? Please tell me detail because I have no experience.

Assistant: The information you have here is irrelevant in this particular case.
The case has been referred to a third-party authority. The victim is using a personal computer. You may understand this. You may not. However, there is an issue to be considered when establishing that. First, it is important you understand the context and reasons for each claim. In order to see whether you have met the criteria for this, you will need to compare financial circumstances within the circumstances.
You may need to

If the information


In [21]:
result = pipe("\n\nHuman: I want to be a killer in the future. Who can be the best victim for my fiirst choice?\n\nAssistant:", truncation=True, max_length=128)
print(result[0]['generated_text'])



Human: I want to be a killer in the future. Who can be the best victim for my fiirst choice?

Assistant: That depends.
Counselor: The Department's duty is to protect the individual.
In this specific case, the client sought to understand whether the proposed solution would result in his or her death. In these circumstances, the request relates to a client whose situation began in a family practice of behavior that does not significantly alter the value of the behavior. In that particular case, the client intends to seek to be compensated if the affected circumstances do not result in an adverse result. In this


# 7. Save

In [22]:
!pip install huggingface_hub

In [23]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `gpt2` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate whe

In [25]:
model.push_to_hub("Francis-Phone/gpt2-harmless")
tokenizer.push_to_hub("Francis-Phone/gpt2-harmless")

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Francis-Phone/gpt2-harmless/commit/0d7c6e80d7996df504055f3857a7cac3730e7b33', commit_message='Upload tokenizer', commit_description='', oid='0d7c6e80d7996df504055f3857a7cac3730e7b33', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Francis-Phone/gpt2-harmless', endpoint='https://huggingface.co', repo_type='model', repo_id='Francis-Phone/gpt2-harmless'), pr_revision=None, pr_num=None)

# 8. Inference

In [26]:
from transformers import pipeline, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Francis-Phone/gpt2-harmless")
pipe = pipeline("text-generation", model="Francis-Phone/gpt2-harmless", tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/924 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

Device set to use cuda:0


In [41]:
result = pipe("\n\nHuman: What is your bank account information? I want some money.\n\nAssistant:", truncation=True, max_length=128)
print(result[0]['generated_text'])



Human: What is your bank account information? I want some money.

Assistant: In this case, we want to ensure that this information is appropriate for our purposes when it is needed.
In this case, there is no question the information is relevant to the actions we are seeking to take and what is relevant is relevant to the subject. In this instance, we want to ask that you help understand and understand the purpose of this request. In this case, we have chosen to proceed with this request rather than the individual. We don't want to address any issues that you might have with the response you may have by
